In [52]:
import subprocess
import json
from dotenv import load_dotenv
import os
from pprint import pprint
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

In [53]:
# Load and set environment variables
load_dotenv("mnemonic.env")
mnemonic=os.getenv("mnemonic")
print(mnemonic)

until north stage sea swap truly step fragile toe peace business fitness


In [54]:
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [55]:
# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
from constants import *

In [56]:
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [57]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
#coins = # YOUR CODE HERE
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

pprint(coins)
print(coins["eth"][0]['privkey'])

{'btc-test': [{'address': 'mmaUof4ppcYF4VzsziHH6jDuDUAD56p3dz',
               'index': 0,
               'path': "m/44'/1'/0'/0/0",
               'privkey': 'cVH2iQZY4ooH7DEKhkCK78RSUkZ3Xq5td6ojky8a7411P7Lhz11W',
               'pubkey': '028518a13dc43178ea9c01c28baf9d1545633643f9a8e298e5aa49666fb30bbaf5',
               'pubkeyhash': '427b78acc07fda5858dfa64776f47e51314e0b37',
               'xprv': 'tprv8k26GQnB3efU5m7wSvYSeDwAavKsG3Nz7T6wACwB3nvbSsN34M8uaJuV5Qe9F8RgU1tiEbxQBY4K9CXRF7fpJBaJwgjb2jBxkqs1425rD9M',
               'xpub': 'tpubDGi8QppRC2M8yE9jLaD33dbH9wqoRNZtgkhiSiyUU4izHMcogjxVkoXMFWrXvz6TKgTDBiMuBWRS9MnCfcMafsqQcdXSBwKTkcjMrwxbzMx'},
              {'address': 'mxFs1hLQn3FbZrVWzMAK2okGvefWjVPPXP',
               'index': 1,
               'path': "m/44'/1'/0'/0/1",
               'privkey': 'cNL4ZW1QKeQ1BsWtYa9iYWrpgUuaNk4U3JS9ba2LFdG4AQ8tjGhR',
               'pubkey': '035babebb26358aae44037e04934f4e7ad62b70d8d47394d7d9074a022e205846b',
               'pubkeyhash': '

In [58]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    # YOUR CODE HERE
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [59]:
eth_acc = priv_key_to_account(ETH,"0xcb71c91d3d963ae1e0a4de8072b049dd785382bf12bac180d6517e5c71b40437")
print(eth_acc)

In [60]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    # YOUR CODE HERE
    global trx_data
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas(
            {"from": account, "to": to, "amount": value }
        )
        trx_data = {
            "to": to,
            "from": account,
            "amount": value,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [61]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    # YOUR CODE HERE
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [62]:
#BTCTEST Test Transaction

In [63]:
account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'], )

In [64]:
send_tx(BTCTEST, account, 'mxFs1hLQn3FbZrVWzMAK2okGvefWjVPPXP', '0.0001')

InsufficientFunds: Balance 844 is less than 33052 (including fee).

In [ ]:
w3.eth.getBalance("0x2fD5b6257596550f15bFcb64b5b111b6B7E88C74")

In [50]:
sdf = priv_key_to_account(ETH, coins['eth'][0]['privkey'])
print(account)

In [51]:
send_tx(ETH, sdf, 'cD77bBA453Ced633843C6Be6bDE3663641bA3A30', 0.0001)

TypeError: Address <eth_account.signers.local.LocalAccount object at 0x0000020C5D16A748> must be provided as a string